# Document Loaders

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [3]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./bella_vista.txt")
docs = loader.load()

In [4]:
print(docs)
print(len(docs))

[Document(page_content="Q: What are the hours of operation for Bella Vista?\nA: Bella Vista is open from 11 a.m. to 11 p.m. from Monday to Saturday. On Sundays, we welcome guests from 12 p.m. to 10 p.m.\n\nQ: What type of cuisine does Bella Vista serve?\nA: Bella Vista offers a delightful blend of Mediterranean and contemporary American cuisine. We pride ourselves on using the freshest ingredients, many of which are sourced locally.\n\nQ: Do you offer vegetarian or vegan options at Bella Vista?\nA: Absolutely! Bella Vista boasts a diverse menu that includes a variety of vegetarian and vegan dishes. Our chefs are also happy to customize dishes based on dietary needs.\n\nQ: Is Bella Vista family-friendly?\nA: Yes, Bella Vista is a family-friendly establishment. We have a dedicated kids' menu and offer high chairs and booster seats for our younger guests.\n\nQ: Can I book private events at Bella Vista?\nA: Certainly! Bella Vista has a private dining area perfect for events, parties, or co

In [5]:
from langchain.schema import Document

example_doc = Document(page_content="test", metadata={"important_info": "hi there"})
example_doc

Document(page_content='test', metadata={'important_info': 'hi there'})

Texts are not loaded 1:1 into the database, but in pieces, so called "chunks". You can define the chunk size and the overlap between the chunks.

To create multiple documents (chunks), you can use a text splitter.

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
)
documents = text_splitter.split_documents(docs)

In [7]:
for doc in documents:
    print(doc)

len(documents)

page_content='Q: What are the hours of operation for Bella Vista?' metadata={'source': './bella_vista.txt'}
page_content='A: Bella Vista is open from 11 a.m. to 11 p.m. from Monday to Saturday. On Sundays, we welcome' metadata={'source': './bella_vista.txt'}
page_content='Sundays, we welcome guests from 12 p.m. to 10 p.m.' metadata={'source': './bella_vista.txt'}
page_content='Q: What type of cuisine does Bella Vista serve?' metadata={'source': './bella_vista.txt'}
page_content='A: Bella Vista offers a delightful blend of Mediterranean and contemporary American cuisine. We' metadata={'source': './bella_vista.txt'}
page_content='cuisine. We pride ourselves on using the freshest ingredients, many of which are sourced locally.' metadata={'source': './bella_vista.txt'}
page_content='Q: Do you offer vegetarian or vegan options at Bella Vista?' metadata={'source': './bella_vista.txt'}
page_content='A: Absolutely! Bella Vista boasts a diverse menu that includes a variety of vegetarian and veg

22